This baseline simply passes in our CAPTCHA dataset, and sees how accurately a well-established OCR library can identify it. This will be used as our baseline model for our machine learning solution for CAPTCHA breaking.

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

In [1]:
import pytesseract
import os

# Mount Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# Sets Kaggle Config Environment Variable (Path of Kaggle.json file)
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Captcha_Datasets"
%cd /content/gdrive/MyDrive/Captcha_Datasets

/content/gdrive/MyDrive/Captcha_Datasets


In [41]:
# Fetch a CAPTCHA Dataset
!kaggle datasets download -d fanbyprinciple/captcha-images

# Then Unzip the Dataset
!unzip \*.zip  && rm *.zip

Streaming output truncated to the last 5000 lines.
  inflating: captcha_images/HVHF.png  
  inflating: captcha_images/HVP8.png  
  inflating: captcha_images/HVST.png  
  inflating: captcha_images/HVV8.png  
  inflating: captcha_images/HVXC.png  
  inflating: captcha_images/HVXJ.png  
  inflating: captcha_images/HW3K.png  
  inflating: captcha_images/HW77.png  
  inflating: captcha_images/HW9K.png  
  inflating: captcha_images/HWA7.png  
  inflating: captcha_images/HWAP.png  
  inflating: captcha_images/HWEU.png  
  inflating: captcha_images/HWG5.png  
  inflating: captcha_images/HWG9.png  
  inflating: captcha_images/HWGW.png  
  inflating: captcha_images/HWN8.png  
  inflating: captcha_images/HWRY.png  
  inflating: captcha_images/HWWQ.png  
  inflating: captcha_images/HWZ4.png  
  inflating: captcha_images/HX24.png  
  inflating: captcha_images/HX5X.png  
  inflating: captcha_images/HX9J.png  
  inflating: captcha_images/HXJ4.png  
  inflating: captcha_images/HXKF.png  
  inflating: 

In [11]:
import cv2 as cv
from PIL import Image
import numpy as np

In [15]:
# Iterate over all files
for files in os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images"):
  if files.endswith(".png"):    
    res = cv.imread("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images/" + files, 0)
    kernel = np.ones((5,5), np.uint8)
    
    #Operations on the image to cleanup and remove noise
    gaussian_blur = cv.GaussianBlur(res,(5,5),0)
    ret, thresh = cv.threshold(gaussian_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    thresh = cv.erode(thresh, kernel, iterations=2)
    thresh = cv.dilate(thresh, kernel, iterations=2)
    thresh = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=2)
    cv.imwrite("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images/processed/" + files, res)   
    

In [25]:
# Setup accuracy tracker
total_images = 0;
correct_images = 0;

print("Beginning OCR...")
for files in os.listdir("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images/processed"):
  if files.endswith(".png"):
    correct_label = files.split('.')[0]

    image = Image.open("/content/gdrive/MyDrive/Captcha_Datasets/captcha_images/processed/" + files)
    # Make pytesseract prediction
    prediction = pytesseract.image_to_string(image)
    if prediction.strip().upper() == correct_label.strip().upper():
      correct_images += 1
    total_images += 1

accuracy = (float(correct_images)/total_images) * 100
print("Total Baseline Model Accuracy: {}%".format(accuracy))

Beginning OCR...
Total Baseline Model Accuracy: 48.60576923076923%
